# Preprocessing with sklearn pipeline

In [1]:
from warnings import filterwarnings
filterwarnings("ignore")

# Goal is to estimate the Weight of Car based on varius factors

# Step 1 - Data Ingestion

In [2]:
import pandas as pd
df = pd.read_csv("Cars93.csv", na_values = ["", "NA"], keep_default_na=False)
df.head()

,id,Manufacturer,Model,Type,Min.Price,Price,Max.Price,MPG.city,MPG.highway,AirBags,...,Passengers,Length,Wheelbase,Width,Turn.circle,Rear.seat.room,Luggage.room,Weight,Origin,Make
0,1,Acura,Integra,Small,12.9,15.9,18.8,25,31,None,...,5,177,102,68,37,26.5,11.0,2705,non-USA,Acura Integra
1,2,Acura,Legend,Midsize,29.2,33.9,38.7,18,25,Driver & Passenger,...,5,195,115,71,38,30.0,15.0,3560,non-USA,Acura Legend
2,3,Audi,90,Compact,25.9,29.1,32.3,20,26,Driver only,...,5,180,102,67,37,28.0,14.0,3375,non-USA,Audi 90
3,4,Audi,100,Midsize,30.8,37.7,44.6,19,26,NaN,...,6,193,106,70,37,31.0,17.0,3405,non-USA,Audi 100
4,5,BMW,535i,Midsize,23.7,30.0,36.2,22,30,Driver only,...,4,186,109,69,39,27.0,13.0,3640,non-USA,BMW 535i


# Target - Weight

# Step 2 - Perform basic data quality checks

In [3]:
df.duplicated().sum()

np.int64(1)

In [4]:
df = df.drop_duplicates(keep = "first").reset_index(drop = True)
df.shape

(93, 28)

In [7]:
m = df.isna().sum()
m[m > 0]

AirBags            4
Rear.seat.room     2
Luggage.room      11
dtype: int64

In [10]:
card = df.select_dtypes(include = "object").nunique() / len(df)
card

Manufacturer       0.344086
Model              1.000000
Type               0.064516
AirBags            0.032258
DriveTrain         0.032258
Cylinders          0.064516
Man.trans.avail    0.021505
Origin             0.021505
Make               1.000000
dtype: float64

In [11]:
high_card = card[card > 0.9]
high_card

Model    1.0
Make     1.0
dtype: float64

In [12]:
df = df.drop(columns = high_card.index)
df.head()

,id,Manufacturer,Type,Min.Price,Price,Max.Price,MPG.city,MPG.highway,AirBags,DriveTrain,...,Fuel.tank.capacity,Passengers,Length,Wheelbase,Width,Turn.circle,Rear.seat.room,Luggage.room,Weight,Origin
0,1,Acura,Small,12.9,15.9,18.8,25,31,None,Front,...,13.2,5,177,102,68,37,26.5,11.0,2705,non-USA
1,2,Acura,Midsize,29.2,33.9,38.7,18,25,Driver & Passenger,Front,...,18.0,5,195,115,71,38,30.0,15.0,3560,non-USA
2,3,Audi,Compact,25.9,29.1,32.3,20,26,Driver only,Front,...,16.9,5,180,102,67,37,28.0,14.0,3375,non-USA
3,4,Audi,Midsize,30.8,37.7,44.6,19,26,NaN,Front,...,21.1,6,193,106,70,37,31.0,17.0,3405,non-USA
4,5,BMW,Midsize,23.7,30.0,36.2,22,30,Driver only,Rear,...,21.1,4,186,109,69,39,27.0,13.0,3640,non-USA


# Seperate X and Y(Weight)

In [24]:
X = df.drop(columns = ["id", "Weight"])
Y = df["Weight"]

In [25]:
X.head()

,Manufacturer,Type,Min.Price,Price,Max.Price,MPG.city,MPG.highway,AirBags,DriveTrain,Cylinders,...,Man.trans.avail,Fuel.tank.capacity,Passengers,Length,Wheelbase,Width,Turn.circle,Rear.seat.room,Luggage.room,Origin
0,Acura,Small,12.9,15.9,18.8,25,31,None,Front,4,...,Yes,13.2,5,177,102,68,37,26.5,11.0,non-USA
1,Acura,Midsize,29.2,33.9,38.7,18,25,Driver & Passenger,Front,6,...,Yes,18.0,5,195,115,71,38,30.0,15.0,non-USA
2,Audi,Compact,25.9,29.1,32.3,20,26,Driver only,Front,6,...,Yes,16.9,5,180,102,67,37,28.0,14.0,non-USA
3,Audi,Midsize,30.8,37.7,44.6,19,26,NaN,Front,6,...,Yes,21.1,6,193,106,70,37,31.0,17.0,non-USA
4,BMW,Midsize,23.7,30.0,36.2,22,30,Driver only,Rear,4,...,Yes,21.1,4,186,109,69,39,27.0,13.0,non-USA


In [26]:
Y.head()

0    2705
1    3560
2    3375
3    3405
4    3640
Name: Weight, dtype: int64

# Train test split

In [27]:
from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.2, random_state=42)

In [28]:
xtrain.head()

,Manufacturer,Type,Min.Price,Price,Max.Price,MPG.city,MPG.highway,AirBags,DriveTrain,Cylinders,...,Man.trans.avail,Fuel.tank.capacity,Passengers,Length,Wheelbase,Width,Turn.circle,Rear.seat.room,Luggage.room,Origin
65,Nissan,Van,16.7,19.1,21.5,17,23,None,Front,6,...,No,20.0,7,190,112,74,41,27.0,NaN,non-USA
15,Chevrolet,Van,14.7,16.3,18.0,18,23,None,Front,6,...,No,20.0,7,178,110,74,44,30.5,NaN,USA
68,Oldsmobile,Midsize,14.2,16.3,18.4,23,31,Driver only,Front,4,...,No,16.5,5,190,105,70,42,28.0,16.0,USA
78,Saturn,Small,9.2,11.1,12.9,28,38,Driver only,Front,4,...,Yes,12.8,5,176,102,68,40,26.5,12.0,USA
30,Ford,Small,6.9,7.4,7.9,31,33,None,Front,4,...,Yes,10.0,4,141,90,63,33,26.0,12.0,USA


In [29]:
ytrain.head()

65    4100
15    3715
68    2890
78    2495
30    1845
Name: Weight, dtype: int64

In [30]:
xtest.head()

,Manufacturer,Type,Min.Price,Price,Max.Price,MPG.city,MPG.highway,AirBags,DriveTrain,Cylinders,...,Man.trans.avail,Fuel.tank.capacity,Passengers,Length,Wheelbase,Width,Turn.circle,Rear.seat.room,Luggage.room,Origin
40,Honda,Sporty,17.0,19.8,22.7,24,31,Driver & Passenger,Front,4,...,Yes,15.9,4,175,100,70,39,23.5,8.0,non-USA
22,Dodge,Small,7.9,9.2,10.6,29,33,None,Front,4,...,Yes,13.2,5,174,98,66,32,26.5,11.0,USA
55,Mazda,Van,16.6,19.1,21.7,18,24,None,4WD,6,...,No,19.6,7,190,110,72,39,27.5,NaN,non-USA
72,Pontiac,Small,8.2,9.0,9.9,31,41,None,Front,4,...,Yes,13.2,4,177,99,66,35,25.5,17.0,USA
0,Acura,Small,12.9,15.9,18.8,25,31,None,Front,4,...,Yes,13.2,5,177,102,68,37,26.5,11.0,non-USA


In [31]:
ytest.head()

40    2865
22    2270
55    3735
72    2350
0     2705
Name: Weight, dtype: int64

# Apply Preprocessing on X

In [32]:
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [34]:
num_cols = X.select_dtypes(include ="number").columns
num_cols

Index(['Min.Price', 'Price', 'Max.Price', 'MPG.city', 'MPG.highway',
       'EngineSize', 'Horsepower', 'RPM', 'Rev.per.mile', 'Fuel.tank.capacity',
       'Passengers', 'Length', 'Wheelbase', 'Width', 'Turn.circle',
       'Rear.seat.room', 'Luggage.room'],
      dtype='object')

In [35]:
cat_cols = X.select_dtypes(include = "object").columns
cat_cols

Index(['Manufacturer', 'Type', 'AirBags', 'DriveTrain', 'Cylinders',
       'Man.trans.avail', 'Origin'],
      dtype='object')

In [40]:
num_pipe = make_pipeline(
    SimpleImputer(strategy="median"),
    StandardScaler()
)

In [41]:
cat_pipe = make_pipeline(
    SimpleImputer(strategy= "most_frequent"),
    OneHotEncoder(handle_unknown="ignore", sparse_output = False, drop="first")
)

In [42]:
pre = ColumnTransformer(
    [
        ("num", num_pipe, num_cols),
        ("cat", cat_pipe, cat_cols)
    ]
).set_output(transform = "pandas")

In [43]:
pre.fit(xtrain)

,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contains sparse matrices,these will be stacked as a sparse matrix if the overall density islower than this value. Use ``sparse_threshold=0`` to always returndense. When the transformed output consists of all dense data, thestacked result will be dense, and this keyword will be ignored.",0.3
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.",None
,"transformer_weights transformer_weights: dict, default=NoneMultiplicative weights for features per transformer. The output of thetransformer is multiplied by these weights. Keys are transformer names,values the weights.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each transformer will beprinted as it is completed.",False
,"verbose_feature_names_out verbose_feature_names_out: bool, str or Callable[[str, str], str], default=True- If True, :meth:`ColumnTransformer.get_feature_names_out` will prefix all feature names with the name of the transformer that generated that feature. It is equivalent to setting `verbose_feature_names_out=""{transformer_name}__{feature_name}""`.- If False, :meth:`ColumnTransformer.get_feature_names_out` will not prefix any feature names and will error if feature names are not unique.- If ``Callable[[str, str], str]``, :meth:`ColumnTransformer.get_feature_names_out` will rename all the features using the name of the transformer. The first argument of the callable is the transformer name and the second argument is the feature name. The returned string will be the new feature name.- If ``str``, it must be a string ready for formatting. The given string will be formatted using two field names: ``transformer_name`` and ``feature_name``. e.g. `

In [45]:
xtrain_pre = pre.transform(xtrain)
xtrain_pre.head()

,num__Min.Price,num__Price,num__Max.Price,num__MPG.city,num__MPG.highway,num__EngineSize,num__Horsepower,num__RPM,num__Rev.per.mile,num__Fuel.tank.capacity,...,cat__AirBags_None,cat__DriveTrain_Front,cat__DriveTrain_Rear,cat__Cylinders_4,cat__Cylinders_5,cat__Cylinders_6,cat__Cylinders_8,cat__Cylinders_rotary,cat__Man.trans.avail_Yes,cat__Origin_non-USA
65,-0.045290,-0.042952,-0.039482,-0.926566,-1.134441,0.307021,0.205378,-0.666653,-0.427741,1.004804,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
15,-0.279660,-0.335700,-0.355094,-0.749850,-1.134441,1.093845,0.603669,-0.666653,-1.218799,1.004804,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
68,-0.338253,-0.335700,-0.319024,0.133731,0.378147,-0.479803,-0.654093,0.006826,0.627002,-0.063961,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
78,-0.924178,-0.879377,-0.814985,1.017312,1.701662,-0.774863,-1.178161,-0.329914,-0.258982,-1.193798,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
30,-1.193703,-1.266224,-1.265859,1.547461,0.756294,-1.364981,-1.639341,-0.329914,1.861052,-2.048810,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [46]:
xtest_pre = pre.transform(xtest)
xtest_pre.head()

,num__Min.Price,num__Price,num__Max.Price,num__MPG.city,num__MPG.highway,num__EngineSize,num__Horsepower,num__RPM,num__Rev.per.mile,num__Fuel.tank.capacity,...,cat__AirBags_None,cat__DriveTrain_Front,cat__DriveTrain_Rear,cat__Cylinders_4,cat__Cylinders_5,cat__Cylinders_6,cat__Cylinders_8,cat__Cylinders_rotary,cat__Man.trans.avail_Yes,cat__Origin_non-USA
40,-0.010135,0.030236,0.068728,0.310447,0.378147,-0.381450,0.394042,1.017044,1.238753,-0.247178,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
22,-1.076518,-1.078028,-1.022387,1.194028,0.756294,-1.168275,-1.031422,1.353784,2.145833,-1.071653,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
55,-0.057009,-0.042952,-0.021447,-0.749850,-0.945368,0.307021,0.289229,-0.329914,-0.058581,0.882659,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
72,-1.041363,-1.098939,-1.085510,1.547461,2.268882,-1.069922,-1.408751,0.680305,1.818862,-1.071653,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
0,-0.490593,-0.377522,-0.282954,0.487164,0.378147,-0.873216,-0.025212,1.858893,1.312585,-1.071653,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0


# Build the model object
y = B0 + B1 * x1 + B2 * x2 + B3 * x3 + ...

In [47]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(xtrain_pre, ytrain)

,"fit_intercept fit_intercept: bool, default=TrueWhether to calculate the intercept for this model. If setto False, no intercept will be used in calculations(i.e. data is expected to be centered).",True
,"copy_X copy_X: bool, default=TrueIf True, X will be copied; else, it may be overwritten.",True
,"tol tol: float, default=1e-6The precision of the solution (`coef_`) is determined by `tol` whichspecifies a different convergence criterion for the `lsqr` solver.`tol` is set as `atol` and `btol` of :func:`scipy.sparse.linalg.lsqr` whenfitting on sparse training data. This parameter has no effect when fittingon dense data... versionadded:: 1.7",1e-06
,"n_jobs n_jobs: int, default=NoneThe number of jobs to use for the computation. This will only providespeedup in case of sufficiently large problems, that is if firstly`n_targets > 1` and secondly `X` is sparse or if `positive` is setto `True`. ``None`` means 1 unless in a:obj:`joblib.parallel_backend` context. ``-1`` means using allprocessors. See :term:`Glossary ` for more details.",None
,"positive positive: bool, default=FalseWhen set to ``True``, forces the coefficients to be positive. Thisoption is only supported for dense arrays.For a comparison between a linear regression model with positive constraintson the regression coefficients and a linear regression without such constraints,see :ref:`sphx_glr_auto_examples_linear_model_plot_nnls.py`... versionadded:: 0.24",False


In [59]:
model.coef_

array([ 1451.98593536, -2907.19838271,  1762.03881281,  -224.51313847,
         -32.88694122,   -31.21156302,   221.61570431,   -95.15238653,
         -45.01364027,  -145.81534813,   109.31669442,   -11.46791352,
          83.76378683,   201.14314408,   -14.69248882,   -57.27279969,
          70.53420907,   -22.32984663,  -150.78674523,  -421.62553168,
          40.7142687 ,   394.90719852,   105.51916383,    29.91341113,
         134.33140622,   -49.24084954,   526.56950633,   534.75141963,
         218.15571189,  -385.77357541,    88.04916735,  -282.77543861,
          12.08302666,  -783.7464714 ,    -4.34194821,    -4.40759825,
         155.22267216,   -69.28961866,   263.44057463,   -15.39711597,
        -299.50474669,   -36.78835142,    32.21761144,   391.6748307 ,
         188.25156458,   121.11354468,  -185.48799686,  -306.63657691,
        -177.83896961,    49.94136466,  -115.97415566,  -105.28112006,
         204.1889185 ,   191.82187309,   -87.41897318,    48.02542955,
      

In [60]:
model.intercept_

np.float64(3405.5327340815534)

In [49]:
# R2 score on train
model.score(xtrain_pre, ytrain)

0.9962358571591687

In [50]:
# R2 score on test
model.score(xtest_pre, ytest)

0.8211470475134268

# apply the metrics

In [51]:
from sklearn.metrics import (
    root_mean_squared_error,
    mean_absolute_error,
    mean_absolute_percentage_error,
    r2_score
)

In [53]:
def evaluate_model(model, x, y):
    ypred = model.predict(x)
    rmse = root_mean_squared_error(y, ypred)
    mae = mean_absolute_error(y, ypred)
    mape = mean_absolute_percentage_error(y, ypred)
    r2 = r2_score(y, ypred)
    print(f"RMSE : {rmse:.2f}")
    print(f"MAE : {mae:.2f}")
    print(f"MAPE : {mape:.2%}")
    print(f"R2 : {r2:.2%}")

In [56]:
print("Train Results :")
evaluate_model(model, xtrain_pre, ytrain)

Train Results :
RMSE : 36.08
MAE : 25.50
MAPE : 0.82%
R2 : 99.62%


In [58]:
print("Test Results : ")
evaluate_model(model, xtest_pre, ytest)

Test Results : 
RMSE : 243.02
MAE : 194.28
MAPE : 6.59%
R2 : 82.11%
